# Test The Model Deployed from Our Pipeline

In [1]:
import boto3
import sagemaker
import pandas as pd

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)

# Retrieve Endpoint Name  

In [2]:
%store -r step_functions_pipeline_endpoint_name

In [3]:
try:
    step_functions_pipeline_endpoint_name
    print('[OK]')
except NameError:
    print('+++++++++++++++++++++++++++++++')
    print('[ERROR] Please run the notebooks in this section before you continue.')
    print('+++++++++++++++++++++++++++++++')

[OK]


In [4]:
print(step_functions_pipeline_endpoint_name)

training-pipeline-2020-09-26-20-25-01


# Wait For The Endpoint To Be Deployed

In [5]:
client = boto3.client('sagemaker')
waiter = client.get_waiter('endpoint_in_service')
waiter.wait(EndpointName=step_functions_pipeline_endpoint_name)

# Invoke the Endpoint

In [6]:
import json
from sagemaker.tensorflow.model import TensorFlowPredictor

predictor = TensorFlowPredictor(endpoint_name=step_functions_pipeline_endpoint_name,
                                sagemaker_session=sess,
                                model_name='saved_model',
                                model_version=0)

In [7]:
reviews = ["This is great!"]

predicted_classes = predictor.predict(reviews)

for predicted_class, review in zip(predicted_classes, reviews):
    print('[Predicted Star Rating: {}]'.format(predicted_class), review)

[Predicted Star Rating: 5] This is great!


# Delete Endpoints
To save money, we should delete the endpoint.

In [8]:
sm.delete_endpoint(
    EndpointName=step_functions_pipeline_endpoint_name
)

{'ResponseMetadata': {'RequestId': 'f8198dcf-a315-44ba-8c1c-7f20c965186d',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'f8198dcf-a315-44ba-8c1c-7f20c965186d',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Sat, 26 Sep 2020 21:11:51 GMT'},
  'RetryAttempts': 0}}

In [ ]:
%%javascript
Jupyter.notebook.save_checkpoint();
Jupyter.notebook.session.delete();